# Intro<br>
This file is to prepare the data for clustering and anomaly detection. It does:
<ol>
    <li>Make feature columns</li>
    <li>Drop uneeded columns</li>
    <li>Encode columns that could be used later as features</li>
    <li>Group the line entry data into journal documents </li>
    <li>Add in needed columns from the ZOTC data (to be used for evaluation)</li>
    <li>Save the cleaned data</li>
</ol>

# Import data

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
from importlib import reload
import pandas as pd

# sys.path.append(r"E:\enyutan\Document\2025 MJE Advanced Analytics\Ishi")
import _00_util_sql
reload(_00_util_sql)
from _00_util_sql import Conn_ODBC

In [52]:
##### Import data 
sql_db=Conn_ODBC(database="JE_ML_2025")

conn=sql_db.odbc_conn_db_pyodbc()

sql_query=f"""
SELECT * FROM data_ishi_GL_cleaned where AccDocNo in (
SELECT DISTINCT([Invoice Number]) FROM Project_Ishi.dbo.ZOTC_combined WHERE [IS rep] = 'SIEW PENG LAU')
"""

gl = sql_db.odbc_run_sql(conn, sql_query, return_result=True)

sql_query=f"SELECT * FROM [data_ishi_ZOTC_combined]"
zotc = sql_db.odbc_run_sql(conn, sql_query, return_result=True)

sql_query=f"SELECT * FROM [PublicHolidays]"
p_hol = sql_db.odbc_run_sql(conn, sql_query, return_result=True)

conn.close()

In [ ]:
##### Import data 
# sql_db=Conn_ODBC(database="JE_ML_2025")

# conn=sql_db.odbc_conn_db_pyodbc()

# sql_query=f"SELECT * FROM [data_ishi_GL_cleaned_2018]"
# gl = sql_db.odbc_run_sql(conn, sql_query, return_result=True)

# sql_query=f"SELECT * FROM [data_ishi_ZOTC_2018]"
# zotc = sql_db.odbc_run_sql(conn, sql_query, return_result=True)

# sql_query=f"SELECT * FROM [PublicHolidays]"
# p_hol = sql_db.odbc_run_sql(conn, sql_query, return_result=True)

# conn.close()

# Clean GL<br>
Make feature columns, dropping uneeded cols and make line entry data into journal document level data. Data that is not standardized across the journal document line entries are stored in lists.<br><br>
Encoding certain categorical columns to use in feature engineering later

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [4]:
df_processed = gl.copy()

## Make new columns

In [5]:
def count_trailing_zeros(cost):
    cost = str(cost)  # Always format to 2 decimal places
    if '.' in cost:
        integer_part, decimal_part = cost.split('.')
        
        count = 0
        for ch in reversed(integer_part):
            if ch == '0':
                count += 1
            else:
                break  # Stop counting once a non-zero is found
        return count
    else:
        return 0
    
df_processed['Amt_DocCurrency_NoOfTrailingZeroes'] = df_processed['Amt_DocCurrency'].apply(count_trailing_zeros)

In [6]:
df_processed['CashDiscountPercentage'] = np.where(
    df_processed['Amt_localCurrency'] != 0,                                        # condition
    df_processed['CashDiscountAmountInLocalCurrency'] / df_processed['Amt_localCurrency'],   # If condition is true
    0  # or np.nan if preferred, if condition is false
)

In [7]:
df_processed.columns.tolist()

['GL_AccType',
 'AccountTypeDesc',
 'GL_AccType_AccountTypeDesc',
 'GL_AccNo',
 'GL_DESCRIPTION',
 'Dr_Cr',
 'Amt_localCurrencyFormatted',
 'AccDocNo',
 'FiscalYear',
 'ClearingDate',
 'DocNoOfClearingDoc',
 'ClearingEntryDate',
 'AccNo_Vendor_Creditor',
 'VendorName',
 'PurchasingDocNo',
 'CustNo',
 'CustName',
 'Document Type Description',
 'MaterialNumber',
 'Quantity',
 'BaseUnitOfMeasure',
 'Qty_PO_PriceUnit',
 'OrderPriceUnit_Purchasing',
 'CashDiscountDays1',
 'CashDiscountDays2',
 'CashDiscountPercentage1',
 'CashDiscountPercentage2',
 'RefDocNo',
 'ReverseDocNo',
 'ReverseDocFiscalYear',
 'TimeOfEntry',
 'Username',
 'DocType',
 'DocumentDateInDoc',
 'PostingDateInDoc',
 'FiscalPeriod',
 'DateOfAccDocEntry',
 'TransactionCode',
 'DocHeaderText',
 'CurrencyKey',
 'ExchangeRate',
 'IsDocPostedToAPreviousPeriod',
 'BusinessTransaction',
 'RefProcedure',
 'GLAccAmtEnteredExcludingTax',
 'IsDocReversalOrReversedDoc',
 'LedgerGroup',
 'NoOfLineItemInAccDoc',
 'IdOfLineItem',
 'Posti

### Date Time features

In [8]:
import calendar
from datetime import datetime, timedelta

def get_last_friday_from_date(date_obj):
    # Extract year and month from the input date
    year = date_obj.year
    month = date_obj.month
    
    # Find the last day of the month
    last_day = calendar.monthrange(year, month)[1]
    last_date = datetime(year, month, last_day)
    
    # Move backwards to find the last Friday (weekday 4)
    while last_date.weekday() != 4:
        last_date -= timedelta(days=1)
    
    return last_date

# Ensure the datetime column is parsed correctly
def make_new_date_cols(df):
    df['DateOfAccDocEntry_and_TimeOfEntry (MYT)'] = pd.to_datetime(df['DateOfAccDocEntry_and_TimeOfEntry (MYT)'])

    # Extract date/time parts
    df['EntryDate'] = df['DateOfAccDocEntry_and_TimeOfEntry (MYT)'].dt.date
    df['EntryTime'] = df['DateOfAccDocEntry_and_TimeOfEntry (MYT)'].dt.time
    df['WeekDayName'] = df['DateOfAccDocEntry_and_TimeOfEntry (MYT)'].dt.day_name()
    df['EntryHour'] = df['DateOfAccDocEntry_and_TimeOfEntry (MYT)'].dt.hour

    # Convert EntryDate to datetime so we can use pandas date functions
    df['EntryDate'] = pd.to_datetime(df['EntryDate'])

    # Calculate End of Month
    df['LastDayOfMonth'] = df['EntryDate'] + pd.offsets.MonthEnd(0)

    # Calculate Last Friday of the month
    df['LastFridayOfMonth'] = df['EntryDate'].apply(get_last_friday_from_date)

    # Start and End of Quarter
    df['StartOfQuarter'] = df['EntryDate'].dt.to_period('Q').dt.start_time
    df['EndOfQuarter'] = df['EntryDate'].dt.to_period('Q').dt.end_time

    # Flag Calculations
    df['IND_MonthEnd'] = (
        df['EntryDate'] == df['LastFridayOfMonth']
    ).astype(int)

    df['LastWeek1'] = (df['EndOfQuarter'] - pd.Timedelta(days=6)).dt.date

    df['IND_QuarterEnd'] = (
        df['EntryDate'] >= df['LastWeek1']
    ).astype(int)

    df['IND_YearEnd'] = (df['EntryDate'].dt.month == 3).astype(int)

    df['IND_Weekend'] = df['WeekDayName'].isin(['Saturday', 'Sunday']).astype(int)

    # Public Holidays
    # public_holidays['HolidayDate'] = pd.to_datetime(public_holidays['HolidayDate'])
    # holiday_dates = set(public_holidays['HolidayDate'])
    holiday_dates = set(p_hol['HolidayDate'])
    df['IND_PublicHol'] = df['EntryDate'].isin(holiday_dates).astype(int)

    # Work Hours (before 8am or after 7pm)
    df['IND_OutsideWorkHours'] = ((df['EntryHour'] < 8) | (df['EntryHour'] >= 19)).astype(int)

    # Optional: Drop intermediate columns if not needed
    df.drop(columns=[
        'LastDayOfMonth', 'LastFridayOfMonth', 'StartOfQuarter', 'EndOfQuarter', 'LastWeek1'
    ], inplace=True, errors='ignore')

    return df

In [9]:
make_new_date_cols(df_processed)

D:\victoriaquek\AppData\Local\Temp\9\ipykernel_58708\4038697045.py:61: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['IND_PublicHol'] = df['EntryDate'].isin(holiday_dates).astype(int)


,GL_AccType,AccountTypeDesc,GL_AccType_AccountTypeDesc,GL_AccNo,GL_DESCRIPTION,Dr_Cr,Amt_localCurrencyFormatted,AccDocNo,FiscalYear,ClearingDate,...,EntryDate,EntryTime,WeekDayName,EntryHour,IND_MonthEnd,IND_QuarterEnd,IND_YearEnd,IND_Weekend,IND_PublicHol,IND_OutsideWorkHours
0,Revenue,G\L Account,Revenue - G\L Account,400000,Third Party Sales,Cr,-10800.00,9902502690,2014,None,...,2014-12-15,16:11:57,Monday,16,0,0,0,0,0,0
1,Revenue,G\L Account,Revenue - G\L Account,400000,Third Party Sales,Cr,-1348.69,9902593470,2014,None,...,2015-01-02,13:19:31,Friday,13,0,0,0,0,0,0
2,Revenue,G\L Account,Revenue - G\L Account,400000,Third Party Sales,Cr,-129.00,9902576266,2014,None,...,2015-01-03,16:02:52,Saturday,16,0,0,0,1,0,0
3,Revenue,G\L Account,Revenue - G\L Account,400000,Third Party Sales,Cr,-6940.00,9902581598,2014,None,...,2014-12-31,09:17:06,Wednesday,9,0,1,0,0,0,0
4,Revenue,G\L Account,Revenue - G\L Account,400000,Third Party Sales,Cr,-5219.00,9902566338,2014,None,...,2014-12-24,18:59:28,Wednesday,18,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111843,Revenue,G\L Account,Revenue - G\L Account,400000,Third Party Sales,Dr,1059.88,9904320818,2015,None,...,2015-11-21,12:29:24,Saturday,12,0,0,0,1,0,0
111844,Revenue,G\L Account,Revenue - G\L Account,400000,Third Party Sales,Dr,894.84,9904320818,2015,None,...,2015-11-21,12:29:24,Saturday,12,0,0,0,1,0,0
111845,Revenue,G\L Account,Revenue - G\L Account,400000,Third Party Sales,Cr,-323.30,9904320818,2015,None,...,2015-11-21,12:29:24,Saturday,12,0,0,0,1,0,0
111846,Revenue,G\L Account,Revenue - G\L Account,400000,Third Party Sales,Dr,323.30,9904320818,2015,None,...,2015-11-21,12:29:24,Saturday,12,0,0,0,1,0,0


## Dropping uneeded cols
Cols not needed for comparison to original database or to use as features are dropped (hardcoded)

In [10]:
must_keep = [
'GL_DESCRIPTION',
'Dr_Cr',
'Amt_localCurrencyFormatted',
'AccDocNo',
'FiscalYear',
'DateOfAccDocEntry',
'NoOfLineItemInAccDoc',
]

# Cols that are not greyed out
impt_cols = [
'GL_DESCRIPTION',
'Dr_Cr',
'Amt_localCurrencyFormatted',
'AccDocNo',
'FiscalYear',
'Document Type Description',
'Quantity',
'Username',
'DateOfAccDocEntry',
'TransactionCode',
'DocHeaderText',
'IsDocReversalOrReversedDoc',
'NoOfLineItemInAccDoc',
'IdOfLineItem',
'PostingKey',
'isLineItemAutomaticallyCreated',
'TermsOfPayment',
'CashDiscountAmountInLocalCurrency',
'PaymentMethod',
'FollowOnDocType',
'Plant',
'hasSubsequentDrCrMemo',
'WBSElement',
'SpecialGLIndicator',
'AccountTypeDesc',
'GL_AccType',
'Username Type',
'Amt_DocCurrency_NoOfTrailingZeroes',
'CashDiscountPercentage',
'RefKeyForLineItem',       # For checking of overpayment

'GL_AccType_AccountTypeDesc',
'DateOfAccDocEntry_and_TimeOfEntry (MYT)',
'IND_MonthEnd',
'IND_QuarterEnd',
'IND_YearEnd',
'IND_Weekend',
'IND_PublicHol',
'IND_OutsideWorkHours',
]

In [11]:
# Remove columns with just one unique value or all NULL
to_drop = []
def remove_cols(df):
    for col in df:
        if col not in impt_cols:
            to_drop.append(col)
        # if df[col].nunique() <= 1 and not df[col].isnull().any():      # add null check because some columns has null and 1 unique value
            # to_drop.append(col)
    
    df.drop(columns=to_drop, inplace=True)   # Edit df directly
    return df

remove_cols(df_processed)

,GL_AccType,AccountTypeDesc,GL_AccType_AccountTypeDesc,GL_DESCRIPTION,Dr_Cr,Amt_localCurrencyFormatted,AccDocNo,FiscalYear,Document Type Description,Quantity,...,DateOfAccDocEntry_and_TimeOfEntry (MYT),Username Type,Amt_DocCurrency_NoOfTrailingZeroes,CashDiscountPercentage,IND_MonthEnd,IND_QuarterEnd,IND_YearEnd,IND_Weekend,IND_PublicHol,IND_OutsideWorkHours
0,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-10800.00,9902502690,2014,Billing Doc.Transfer,0.0,...,2014-12-15 16:11:57,MY,2,0.0,0,0,0,0,0,0
1,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-1348.69,9902593470,2014,Billing Doc.Transfer,0.0,...,2015-01-02 13:19:31,MY,0,0.0,0,0,0,0,0,0
2,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-129.00,9902576266,2014,Billing Doc.Transfer,0.0,...,2015-01-03 16:02:52,MY,0,0.0,0,0,0,1,0,0
3,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-6940.00,9902581598,2014,Billing Doc.Transfer,0.0,...,2014-12-31 09:17:06,MY,1,0.0,0,1,0,0,0,0
4,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-5219.00,9902566338,2014,Billing Doc.Transfer,0.0,...,2014-12-24 18:59:28,MY,0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111843,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Dr,1059.88,9904320818,2015,Billing Doc.Transfer,0.0,...,2015-11-21 12:29:24,MY,0,0.0,0,0,0,1,0,0
111844,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Dr,894.84,9904320818,2015,Billing Doc.Transfer,0.0,...,2015-11-21 12:29:24,MY,0,0.0,0,0,0,1,0,0
111845,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-323.30,9904320818,2015,Billing Doc.Transfer,0.0,...,2015-11-21 12:29:24,MY,0,0.0,0,0,0,1,0,0
111846,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Dr,323.30,9904320818,2015,Billing Doc.Transfer,0.0,...,2015-11-21 12:29:24,MY,0,0.0,0,0,0,1,0,0


In [12]:
print(f'Number of cols dropped:{len(to_drop)}')
print('Cols dropped')
print(to_drop)
# Should only drop columns with 1 unique value and no nulls

Number of cols dropped:86
Cols dropped
['GL_AccNo', 'ClearingDate', 'DocNoOfClearingDoc', 'ClearingEntryDate', 'AccNo_Vendor_Creditor', 'VendorName', 'PurchasingDocNo', 'CustNo', 'CustName', 'MaterialNumber', 'BaseUnitOfMeasure', 'Qty_PO_PriceUnit', 'OrderPriceUnit_Purchasing', 'CashDiscountDays1', 'CashDiscountDays2', 'CashDiscountPercentage1', 'CashDiscountPercentage2', 'RefDocNo', 'ReverseDocNo', 'ReverseDocFiscalYear', 'TimeOfEntry', 'DocType', 'DocumentDateInDoc', 'PostingDateInDoc', 'FiscalPeriod', 'CurrencyKey', 'ExchangeRate', 'IsDocPostedToAPreviousPeriod', 'BusinessTransaction', 'RefProcedure', 'GLAccAmtEnteredExcludingTax', 'LedgerGroup', 'AccountType', 'TaxOnSalesPurchasesCode', 'Amt_localCurrency', 'Amt_DocCurrency', 'Amt_UpdateGL', 'Currency_UpdateGL', 'TaxType', 'TransactionKey', 'ValueDate', 'ItemText', 'GrpAccNo', 'CostCenter', 'BillingDoc', 'PersonnelNo', 'isSalesRelated', 'isResidentGLAcc', 'CanLineItemsBeDisplayedbyAcc', 'isOpenItemMgt', 'isLineItemNotLiableToCashDi

In [13]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111848 entries, 0 to 111847
Data columns (total 38 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   GL_AccType                               111848 non-null  object        
 1   AccountTypeDesc                          111848 non-null  object        
 2   GL_AccType_AccountTypeDesc               111848 non-null  object        
 3   GL_DESCRIPTION                           111848 non-null  object        
 4   Dr_Cr                                    111848 non-null  object        
 5   Amt_localCurrencyFormatted               111848 non-null  float64       
 6   AccDocNo                                 111848 non-null  int64         
 7   FiscalYear                               111848 non-null  int64         
 8   Document Type Description                111848 non-null  object        
 9   Quantity                  

## Encoding


In [14]:
identifiers = ['FiscalYear', 'AccDocNo']
non_features = ['Dr_Cr','FiscalYear', 'AccDocNo', 'DateOfAccDocEntry', 'DateOfAccDocEntry_and_TimeOfEntry (MYT)', 
'NoOfLineItemInAccDoc','RefKeyForLineItem']
zotc_cols = ['Customer PO Number', 'IS Rep', 'Sold-To Id', 'OS Rep']

In [15]:
# Make function to encode selected columns
to_encode = []
others = []
encoded_cols = {}

def encode_col(df):
    
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    categorical_cols = [item for item in categorical_cols if item not in non_features]

    for col in categorical_cols:
        # print(col)
        if df[col].astype(str).nunique() <= 45 and df[col].astype(str).nunique() >= 2:    
            # Columns with only 1 unique value or 0 ie all null are ignored
            to_encode.append(col)
        else:
            others.append(col)

    print(to_encode)

    for col in to_encode:
        le = LabelEncoder()
        df[col] = df[col].apply(lambda x: str(x) if pd.notnull(x) else x)
        df['labelled_'+col] = le.fit_transform(df[col])
        encoded_cols[col] = le

    return df

encode_col(df_processed)
# print(df_processed.head(3))  
# 2 mins

# Print label mappings
print("\nLabel Mappings:")
for col, encoder in encoded_cols.items():
    print(f"\n{col}:")
    for i, label in enumerate(encoder.classes_):
        print(f"  {label} => {i}")

['GL_AccType', 'AccountTypeDesc', 'GL_AccType_AccountTypeDesc', 'GL_DESCRIPTION', 'Document Type Description', 'Username', 'TransactionCode', 'IdOfLineItem', 'PostingKey', 'isLineItemAutomaticallyCreated', 'TermsOfPayment', 'PaymentMethod', 'FollowOnDocType', 'Plant', 'Username Type']

Label Mappings:

GL_AccType:
  Asset => 0
  Cost of Goods Sold (COGS) => 1
  Liability => 2
  Other Costs => 3
  Other Revenue => 4
  Revenue => 5

AccountTypeDesc:
  Customer => 0
  G\L Account => 1

GL_AccType_AccountTypeDesc:
  Asset - Customer => 0
  Asset - G\L Account => 1
  Cost of Goods Sold (COGS) - G\L Account => 2
  Liability - G\L Account => 3
  Other Costs - G\L Account => 4
  Other Revenue - G\L Account => 5
  Revenue - G\L Account => 6

GL_DESCRIPTION:
  ACOP Returns Resrve => 0
  ACOP Transit - IM Funded => 1
  ACOP Transit - Standard => 2
  ACOP Transit - Vendor Funded => 3
  ACOPs # RETURNS => 4
  Accounts Receivable Control Account => 5
  Cos ACOP Adjustments => 6
  Cos Sales, Ret And 

## Entries to JE<br>
Remove entries that have 0 Amt_localCurrencyFormatted and are GL_Description = "Realized losses forex transactions" or "Realized gains forex transactions"

In [16]:
# Define the descriptions to filter
target_descriptions = {
    'Realized Losses Forex Transactions',
    'Realised Gains Forex Transactions'
}

# Filter out rows that match both conditions
df_filtered = df_processed[~(                                       # ~ (tilde) selects rows that do not match a certain condition
    df_processed['GL_DESCRIPTION'].isin(target_descriptions) &
    (df_processed['Amt_localCurrencyFormatted'] == 0)
)]
df_filtered

,GL_AccType,AccountTypeDesc,GL_AccType_AccountTypeDesc,GL_DESCRIPTION,Dr_Cr,Amt_localCurrencyFormatted,AccDocNo,FiscalYear,Document Type Description,Quantity,...,labelled_Username,labelled_TransactionCode,labelled_IdOfLineItem,labelled_PostingKey,labelled_isLineItemAutomaticallyCreated,labelled_TermsOfPayment,labelled_PaymentMethod,labelled_FollowOnDocType,labelled_Plant,labelled_Username Type
0,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-10800.00,9902502690,2014,Billing Doc.Transfer,0.0,...,13,0,1,5,1,10,2,2,0,0
1,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-1348.69,9902593470,2014,Billing Doc.Transfer,0.0,...,39,0,1,5,1,10,2,2,0,0
2,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-129.00,9902576266,2014,Billing Doc.Transfer,0.0,...,26,0,1,5,1,10,2,2,0,0
3,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-6940.00,9902581598,2014,Billing Doc.Transfer,0.0,...,26,0,1,5,1,10,2,2,0,0
4,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-5219.00,9902566338,2014,Billing Doc.Transfer,0.0,...,13,0,1,5,1,10,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111843,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Dr,1059.88,9904320818,2015,Billing Doc.Transfer,0.0,...,13,0,1,4,1,10,2,2,0,0
111844,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Dr,894.84,9904320818,2015,Billing Doc.Transfer,0.0,...,13,0,1,4,1,10,2,2,0,0
111845,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Cr,-323.30,9904320818,2015,Billing Doc.Transfer,0.0,...,13,0,1,5,1,10,2,2,0,0
111846,Revenue,G\L Account,Revenue - G\L Account,Third Party Sales,Dr,323.30,9904320818,2015,Billing Doc.Transfer,0.0,...,13,0,1,4,1,10,2,2,0,0


Grouping

In [17]:
features = [
'GL_DESCRIPTION',
'Amt_localCurrencyFormatted',
'Document Type Description',
'Quantity',
'Username',
'TransactionCode',
'DocHeaderText',
'IsDocReversalOrReversedDoc',
'IdOfLineItem',
'PostingKey',
'isLineItemAutomaticallyCreated',
'TermsOfPayment',
'CashDiscountAmountInLocalCurrency',
'PaymentMethod',
'Plant',
'hasSubsequentDrCrMemo',
'WBSElement',
'SpecialGLIndicator',
# 'AccountTypeDesc',
# 'GL_AccType',
'GL_AccType_AccountTypeDesc',
'Username Type',
'Amt_DocCurrency_NoOfTrailingZeroes',
'CashDiscountPercentage',
'IND_MonthEnd',
'IND_QuarterEnd',
'IND_YearEnd',
'IND_Weekend',
'IND_PublicHol',
'IND_OutsideWorkHours',
# 'IND_NonWorkDay',

# New features from ZOTC
# 'Carrier Key Desc'
]

In [18]:
encoded_cols = [col for col in df_filtered.columns if col.startswith('labelled_')]
print(len(encoded_cols))
encoded_cols

15


['labelled_GL_AccType',
 'labelled_AccountTypeDesc',
 'labelled_GL_AccType_AccountTypeDesc',
 'labelled_GL_DESCRIPTION',
 'labelled_Document Type Description',
 'labelled_Username',
 'labelled_TransactionCode',
 'labelled_IdOfLineItem',
 'labelled_PostingKey',
 'labelled_isLineItemAutomaticallyCreated',
 'labelled_TermsOfPayment',
 'labelled_PaymentMethod',
 'labelled_FollowOnDocType',
 'labelled_Plant',
 'labelled_Username Type']

In [19]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111848 entries, 0 to 111847
Data columns (total 53 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   GL_AccType                               111848 non-null  object        
 1   AccountTypeDesc                          111848 non-null  object        
 2   GL_AccType_AccountTypeDesc               111848 non-null  object        
 3   GL_DESCRIPTION                           111848 non-null  object        
 4   Dr_Cr                                    111848 non-null  object        
 5   Amt_localCurrencyFormatted               111848 non-null  float64       
 6   AccDocNo                                 111848 non-null  int64         
 7   FiscalYear                               111848 non-null  int64         
 8   Document Type Description                111848 non-null  object        
 9   Quantity                  

In [20]:
others = non_features + features + encoded_cols

df_sorted = df_filtered.copy()
df_sorted['NoOfLineItemInAccDoc_Num'] = pd.to_numeric(df_sorted['NoOfLineItemInAccDoc'], errors='coerce')
df_sorted = df_sorted.sort_values(by='NoOfLineItemInAccDoc_Num')
df_sorted.drop(columns='NoOfLineItemInAccDoc_Num', inplace=True)

# Create pivot table with multi-index columns
pivot = df_sorted.pivot_table(
    index=identifiers,
    columns=['Dr_Cr', 'GL_AccType_AccountTypeDesc'],
    values='Amt_localCurrencyFormatted',
    aggfunc='sum'
)

# Flatten the column MultiIndex into single strings like "Dr_Asset"
pivot.columns = [f"{col[0]}_{col[1]}" for col in pivot.columns]
pivot.columns

Index(['Cr_Asset - Customer', 'Cr_Asset - G\L Account',
       'Cr_Cost of Goods Sold (COGS) - G\L Account',
       'Cr_Liability - G\L Account', 'Cr_Other Costs - G\L Account',
       'Cr_Other Revenue - G\L Account', 'Cr_Revenue - G\L Account',
       'Dr_Asset - Customer', 'Dr_Asset - G\L Account',
       'Dr_Cost of Goods Sold (COGS) - G\L Account',
       'Dr_Liability - G\L Account', 'Dr_Other Costs - G\L Account',
       'Dr_Other Revenue - G\L Account', 'Dr_Revenue - G\L Account'],
      dtype='object')

In [21]:
for id in identifiers:
    if id in others:
        others.remove(id)

# Group other columns (like GL_Description) into lists
grouped_data = (df_sorted.groupby(identifiers)[others].agg(list).reset_index())

# Merge grouped info back with pivot table
final_df = pd.merge(grouped_data, pivot, on=identifiers, how='left')

# Optional: Fill NA with 0 for numeric columns only
numeric_cols = final_df.select_dtypes(include='number').columns
final_df[numeric_cols] = final_df[numeric_cols].fillna(0)

# Reset index to get flat DataFrame
# pivot.reset_index(inplace=True)

# Optional: Fill missing values with 0 if desired
# pivot = pivot.fillna(0)
# pivot

# 10 mins

In [22]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18992 entries, 0 to 18991
Data columns (total 64 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   FiscalYear                                  18992 non-null  int64  
 1   AccDocNo                                    18992 non-null  int64  
 2   Dr_Cr                                       18992 non-null  object 
 3   DateOfAccDocEntry                           18992 non-null  object 
 4   DateOfAccDocEntry_and_TimeOfEntry (MYT)     18992 non-null  object 
 5   NoOfLineItemInAccDoc                        18992 non-null  object 
 6   RefKeyForLineItem                           18992 non-null  object 
 7   GL_DESCRIPTION                              18992 non-null  object 
 8   Amt_localCurrencyFormatted                  18992 non-null  object 
 9   Document Type Description                   18992 non-null  object 
 10  Quantity  

### Aggreting cols

In [ ]:
# List of cols where data within these cols should be the same across all entries in the JE
can_be_agg_cols = [
'FiscalYear',  # Should alr be the same (not in list)
'AccDocNo',    # Should alr be the same (not in list)
'DateOfAccDocEntry',
'Username',
'IsDocReversalOrReversedDoc', # Check
'TransactionCode',
'DocHeaderText',
'Username Type',
'Document Type Description',

# 'RefKeyForLineItem',       # For checking of overpayment

# 'labelled_IsDocReversalOrReversedDoc',
'labelled_TransactionCode',
'labelled_Username Type',
'labelled_Document Type Description',
'labelled_Carrier Key Desc',

'DateOfAccDocEntry_and_TimeOfEntry (MYT)',     # Used to derive the IND_
'IND_MonthEnd',
'IND_QuarterEnd',
'IND_YearEnd',
'IND_Weekend',
'IND_PublicHol',
'IND_OutsideWorkHours',

# 'IND_NonWorkDay',

# 'Customer PO Number',
# 'IS Rep',
# 'Sold-To Id',
# 'Carrier Key Desc',
# 'OS Rep'
]

In [24]:
def process_cell(value):
    # Skip non-list values
    if not isinstance(value, list):
        return value

    # Normalize: treat None and np.nan as equivalent
    normalized = [
        None if pd.isna(v) else v
        for v in value
    ]

    # If all items are None/Nan, return None
    if all(v is None for v in normalized):
        return None

    # Get unique non-null values
    unique_vals = list(set(v for v in normalized if v is not None))

    # If only one unique non-null value, return it; else keep the list
    if len(unique_vals) == 1:
        return unique_vals[0]
    else:
        return value

In [25]:
import pandas as pd

def process_cell(value):
    """
    Process a cell that may be a list of values.
    - Skips non-lists (returns as-is)
    - Cleans lists: removes NA, dedups, returns single value if one remains, else sorted list
    - Only works for lists of hashable (non-list/dict) values
    """
    # Skip non-list values
    if not isinstance(value, list):
        return value

    # Normalize and filter non-NA values
    cleaned = []
    for v in value:
        if pd.isna(v):
            continue  # skip NaN, None, etc.
        # If item in list is itself a list → convert to string (or skip)
        if isinstance(v, (list, dict, set)):
            v = str(v)  # make it a string to avoid unhashable issues
        cleaned.append(v)

    # Remove duplicates via dict to preserve order (Python 3.7+)
    seen = set()
    unique_vals = []
    for v in cleaned:
        # Use tuple for unhashable types (won't happen now due to str conversion above)
        key = v if isinstance(v, (str, int, float, bool)) else str(v)
        if key not in seen:
            seen.add(key)
            unique_vals.append(v)

    # Decide what to return
    if len(unique_vals) == 0:
        return None
    elif len(unique_vals) == 1:
        return unique_vals[0]  # scalar
    else:
        return unique_vals  # list of scalars

In [26]:
# Check if column data is all the same / the column data can be aggregated to become just 1 value instead of a list eg 0.0 instead of [0.0, 0.0]
def agg_col(df, cols_to_agg):
    for col in cols_to_agg:
        print(f'Aggregating {col}')
        if col not in df.columns:
            print(f"Column '{col}' not found in DataFrame. Skipping...")
            continue

        # Make a copy of the original column
        original_col = df[col].copy()

        # Apply processing to each cell
        processed_col = df[col].apply(process_cell)

        # Check if all values in the processed column are NOT lists
        all_non_list = processed_col.apply(lambda x: not isinstance(x, list)).all()

        if all_non_list:
            # Replace the column with the cleaned version
            df[col] = processed_col
            print(f"✅ Column '{col}' was successfully aggregated and flattened.\n")
        else:
            # Revert back to original if not fully aggregable
            print(f"❌ Column '{col}' could not be fully aggregated. Keeping original.\n")

    return df

agg_col(final_df, can_be_agg_cols)

Aggregating FiscalYear
✅ Column 'FiscalYear' was successfully aggregated and flattened.

Aggregating AccDocNo
✅ Column 'AccDocNo' was successfully aggregated and flattened.

Aggregating DateOfAccDocEntry
✅ Column 'DateOfAccDocEntry' was successfully aggregated and flattened.

Aggregating Username
✅ Column 'Username' was successfully aggregated and flattened.

Aggregating IsDocReversalOrReversedDoc
✅ Column 'IsDocReversalOrReversedDoc' was successfully aggregated and flattened.

Aggregating TransactionCode
✅ Column 'TransactionCode' was successfully aggregated and flattened.

Aggregating DocHeaderText
✅ Column 'DocHeaderText' was successfully aggregated and flattened.

Aggregating Username Type
✅ Column 'Username Type' was successfully aggregated and flattened.

Aggregating Document Type Description
✅ Column 'Document Type Description' was successfully aggregated and flattened.

Aggregating labelled_TransactionCode
✅ Column 'labelled_TransactionCode' was successfully aggregated and flat

,FiscalYear,AccDocNo,Dr_Cr,DateOfAccDocEntry,DateOfAccDocEntry_and_TimeOfEntry (MYT),NoOfLineItemInAccDoc,RefKeyForLineItem,GL_DESCRIPTION,Amt_localCurrencyFormatted,Document Type Description,...,Cr_Other Costs - G\L Account,Cr_Other Revenue - G\L Account,Cr_Revenue - G\L Account,Dr_Asset - Customer,Dr_Asset - G\L Account,Dr_Cost of Goods Sold (COGS) - G\L Account,Dr_Liability - G\L Account,Dr_Other Costs - G\L Account,Dr_Other Revenue - G\L Account,Dr_Revenue - G\L Account
0,2014,9900467949,"[Dr, Cr, Cr, Dr, Cr]",2014-01-08,2014-01-08 20:21:18,"[001, 002, 003, 004, 005]","[5821-MRT-PMO-004, None, None, None, None]","[Accounts Receivable Control Account, Third Pa...","[3018.0, -2848.0, -284.0, 284.0, -170.0]",Billing Doc.Transfer,...,0.0,0.0,-3302.0,3018.0,0.0,0.0,0.0,0.0,0.0,284.0
1,2014,9900546273,"[Dr, Cr]",2014-01-05,2014-01-06 14:23:14,"[001, 002]","[8500008426, None]","[Accounts Receivable Control Account, Third Pa...","[1280.0, -1280.0]",Billing Doc.Transfer,...,0.0,0.0,-1280.0,1280.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,9900555709,"[Dr, Cr]",2014-01-06,2014-01-06 21:30:01,"[001, 002]","[8400061160, None]","[Accounts Receivable Control Account, Third Pa...","[2128.0, -2128.0]",Billing Doc.Transfer,...,0.0,0.0,-2128.0,2128.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,9900588004,"[Dr, Cr, Cr, Cr]",2014-01-10,2014-01-10 17:54:51,"[001, 002, 003, 004]","[13012154, None, None, None]","[Accounts Receivable Control Account, Third Pa...","[201.0, -67.0, -67.0, -67.0]",Billing Doc.Transfer,...,0.0,0.0,-201.0,201.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014,9900634699,"[Dr, Cr]",2014-01-02,2014-01-02 19:09:35,"[001, 002]","[626733, None]","[Accounts Receivable Control Account, Third Pa...","[66.0, -66.0]",Billing Doc.Transfer,...,0.0,0.0,-66.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18987,2019,9950788382,"[Cr, Dr]",2019-06-14,2019-06-14 16:15:56,"[001, 002]","[190240AUT, None]","[Accounts Receivable Control Account, Marketin...","[-20.0, 20.0]",Bill Doc.Tr - Cr Mem,...,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0
18988,2019,9950790111,"[Cr, Dr]",2019-06-18,2019-06-18 18:15:10,"[001, 002]","[180414HPB(R1), None]","[Accounts Receivable Control Account, Vendor P...","[-120.0, 120.0]",Bill Doc.Tr - Cr Mem,...,0.0,0.0,0.0,0.0,0.0,0.0,120.0,0.0,0.0,0.0
18989,2019,9950790176,"[Cr, Dr]",2019-06-18,2019-06-18 18:15:51,"[001, 002]","[180417HPW(R1), None]","[Accounts Receivable Control Account, Vendor P...","[-150.0, 150.0]",Bill Doc.Tr - Cr Mem,...,0.0,0.0,0.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0
18990,2019,9950790623,"[Cr, Dr]",2019-06-18,2019-06-19 10:15:53,"[001, 002]","[180416HPD(R1), None]","[Accounts Receivable Control Account, Vendor P...","[-1700.0, 1700.0]",Bill Doc.Tr - Cr Mem,...,0.0,0.0,0.0,0.0,0.0,0.0,1700.0,0.0,0.0,0.0


In [27]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18992 entries, 0 to 18991
Data columns (total 64 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   FiscalYear                                  18992 non-null  int64         
 1   AccDocNo                                    18992 non-null  int64         
 2   Dr_Cr                                       18992 non-null  object        
 3   DateOfAccDocEntry                           18992 non-null  object        
 4   DateOfAccDocEntry_and_TimeOfEntry (MYT)     18992 non-null  datetime64[ns]
 5   NoOfLineItemInAccDoc                        18992 non-null  object        
 6   RefKeyForLineItem                           18992 non-null  object        
 7   GL_DESCRIPTION                              18992 non-null  object        
 8   Amt_localCurrencyFormatted                  18992 non-null  object        
 9   Docume

#### Checking

In [28]:
def all_lists_have_same_elements(series):
    def is_nan_or_none(val):
        return val is None or (isinstance(val, float) and math.isnan(val))
    
    def is_uniform(lst):
        if not isinstance(lst, list):
            return False  # Not a list -> not uniform

        if not lst:
            return True # Empty list -> uniform
        
        # If all elements are NaN or None, consider it uniform
        if all(is_nan_or_none(elem) for elem in lst):
            return True
        
        # Get first non-null value to compare with others
        first = next((elem for elem in lst if not is_nan_or_none(elem)), None)
        if first is None:
            return True  # List has only NaNs/Nones → already covered, but safe fallback

        return all(elem == first or is_nan_or_none(elem) for elem in lst)

    return series.apply(is_uniform).all()

In [92]:
# print(all_lists_have_same_elements(final_df['IS Rep']))
# print(all_lists_have_same_elements(final_df['OS Rep']))

In [ ]:
def get_non_uniform_rows(series):
    def is_nan_or_none(val):
        return val is None or (isinstance(val, float) and math.isnan(val))
    
    def is_non_uniform(lst):
        if not isinstance(lst, list) or not lst:
            return True  # Empty or invalid lists are considered non-uniform
        
        # If all elements are NaN or None, consider it uniform
        if all(is_nan_or_none(elem) for elem in lst):
            return False

        first = lst[0]
        return any(elem != first for elem in lst if not is_nan_or_none(elem))

    return series[series.apply(is_non_uniform)]

In [ ]:
get_non_uniform_rows(final_df['hasSubsequentDrCrMemo'])

# Merge ZOTC data with GL<br>
Add certain ZOTC columns to Journal document data to use for checking / feature later
<ol>
    <li>IS Rep: Who were behind the journal document -> risk indicator
    <li>Sold-To Id: Customer name
    <li>Customer PO Number: Has WC inside -> risk indicator
</ol>

In [ ]:
##### Import data 
sql_db=Conn_ODBC(database="JE_ML_2025")

conn=sql_db.odbc_conn_db_pyodbc()

sql_query=f"SELECT * FROM [data_ishi_ZOTC_combined]"
zotc = sql_db.odbc_run_sql(conn, sql_query, return_result=True)

conn.close()

In [80]:
zotc.columns

Index(['Sold-To Key', 'Sold-To Id', 'Document Currency', 'Net Product Price',
       'Net Sales', 'Sales Cost', 'ACOP Amount', 'Net Sales Cost',
       'Sales Margin', 'Sales Margin %', 'Quantity Sold', 'Invoice Number',
       'Invoice Line Item', 'Invoice Date', 'Order Number', 'Order Date',
       'Purchase Order Type Desc', 'Order Line Item', 'Billing Type',
       'Document Type Key', 'IS Rep', 'Ship-to Key', 'Ship-to',
       'Customer PO Number', 'Sales Office', 'Sales Group',
       'Terms Of Payment Desc', 'Material Key', 'Material Desc',
       'Manufacturer Part Number', 'Vendor Sub Range', 'Vendor Sub Range Desc',
       'Vendor Key', 'Vendor', 'Order Created By'],
      dtype='object')

In [88]:
zotc_siew = zotc[zotc['IS Rep']=='SIEW PENG LAU'][['IS Rep','Invoice Number', 'Sold-To Id', 'Customer PO Number']]
zotc_siew.rename(columns={'Invoice Number': 'AccDocNo'}, inplace=True)
zotc_siew.head(5)

,IS Rep,AccDocNo,Sold-To Id,Customer PO Number
107,SIEW PENG LAU,9.906155e+09,CTC GLOBAL SDN BHD,8400073808
111,SIEW PENG LAU,9.906155e+09,SISTEM RKK SDN BHD,637622
164,SIEW PENG LAU,9.906155e+09,PIGEON SOLUTION,2016-020
165,SIEW PENG LAU,9.906155e+09,PIGEON SOLUTION,2016-020
166,SIEW PENG LAU,9.906155e+09,PIGEON SOLUTION,2016-020


In [95]:
# Now merge with GL
result = pd.merge(
    final_df,
    zotc_siew,
    on='AccDocNo',
    how='left'
)
result.head()

,FiscalYear,AccDocNo,Dr_Cr,DateOfAccDocEntry,DateOfAccDocEntry_and_TimeOfEntry (MYT),NoOfLineItemInAccDoc,RefKeyForLineItem,GL_DESCRIPTION,Amt_localCurrencyFormatted,Document Type Description,...,Dr_Asset - Customer,Dr_Asset - G\L Account,Dr_Cost of Goods Sold (COGS) - G\L Account,Dr_Liability - G\L Account,Dr_Other Costs - G\L Account,Dr_Other Revenue - G\L Account,Dr_Revenue - G\L Account,IS Rep,Sold-To Id,Customer PO Number
0,2014,9900467949,"[Dr, Cr, Cr, Dr, Cr]",2014-01-08,2014-01-08 20:21:18,"[001, 002, 003, 004, 005]","[5821-MRT-PMO-004, None, None, None, None]","[Accounts Receivable Control Account, Third Pa...","[3018.0, -2848.0, -284.0, 284.0, -170.0]",Billing Doc.Transfer,...,3018.0,0.0,0.0,0.0,0.0,0.0,284.0,SIEW PENG LAU,APEX COMMUNICATIONS SDN BHD,5821-MRT-PMO-004
1,2014,9900467949,"[Dr, Cr, Cr, Dr, Cr]",2014-01-08,2014-01-08 20:21:18,"[001, 002, 003, 004, 005]","[5821-MRT-PMO-004, None, None, None, None]","[Accounts Receivable Control Account, Third Pa...","[3018.0, -2848.0, -284.0, 284.0, -170.0]",Billing Doc.Transfer,...,3018.0,0.0,0.0,0.0,0.0,0.0,284.0,SIEW PENG LAU,APEX COMMUNICATIONS SDN BHD,5821-MRT-PMO-004
2,2014,9900467949,"[Dr, Cr, Cr, Dr, Cr]",2014-01-08,2014-01-08 20:21:18,"[001, 002, 003, 004, 005]","[5821-MRT-PMO-004, None, None, None, None]","[Accounts Receivable Control Account, Third Pa...","[3018.0, -2848.0, -284.0, 284.0, -170.0]",Billing Doc.Transfer,...,3018.0,0.0,0.0,0.0,0.0,0.0,284.0,SIEW PENG LAU,APEX COMMUNICATIONS SDN BHD,5821-MRT-PMO-004
3,2014,9900546273,"[Dr, Cr]",2014-01-05,2014-01-06 14:23:14,"[001, 002]","[8500008426, None]","[Accounts Receivable Control Account, Third Pa...","[1280.0, -1280.0]",Billing Doc.Transfer,...,1280.0,0.0,0.0,0.0,0.0,0.0,0.0,SIEW PENG LAU,CTC GLOBAL SDN BHD,8500008426
4,2014,9900555709,"[Dr, Cr]",2014-01-06,2014-01-06 21:30:01,"[001, 002]","[8400061160, None]","[Accounts Receivable Control Account, Third Pa...","[2128.0, -2128.0]",Billing Doc.Transfer,...,2128.0,0.0,0.0,0.0,0.0,0.0,0.0,SIEW PENG LAU,CTC GLOBAL SDN BHD,8400061160


In [94]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41346 entries, 0 to 41345
Data columns (total 67 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   FiscalYear                                  41346 non-null  int64         
 1   AccDocNo                                    41346 non-null  int64         
 2   Dr_Cr                                       41346 non-null  object        
 3   DateOfAccDocEntry                           41346 non-null  object        
 4   DateOfAccDocEntry_and_TimeOfEntry (MYT)     41346 non-null  datetime64[ns]
 5   NoOfLineItemInAccDoc                        41346 non-null  object        
 6   RefKeyForLineItem                           41346 non-null  object        
 7   GL_DESCRIPTION                              41346 non-null  object        
 8   Amt_localCurrencyFormatted                  41346 non-null  object        
 9   Docume

## Python

In [ ]:
# import pandas as pd

# zotc_1 = zotc.copy()

# # Required columns to bring from ZOTC
# required_cols = ['IS Rep', 'OS Rep', 'Carrier Key Desc', 'Customer PO Number', 'Sold-To Id']

# # Check if all required columns exist in ZOTC
# missing_cols = [col for col in required_cols if col not in zotc_1.columns]
# if missing_cols:
#     print(f"Missing columns in ZOTC: {missing_cols}")

# # Rename 'Invoice Number' to 'AccDocNo' to match GL
# zotc_clean = zotc_1.rename(columns={'Invoice Number': 'AccDocNo'})

# # Ensure AccDocNo is clean scalar type (string)
# def extract_scalar_acc_doc_no(x):
#     """Convert any type to string. If list, take first non-null element."""
#     if isinstance(x, list):
#         # Extract first valid item
#         for item in x:
#             if pd.notna(item):
#                 return str(item).strip()
#         return None  # all were NaN
#     elif pd.isna(x):
#         return None
#     else:
#         return str(x).strip()

# # Apply cleaning
# zotc_clean['AccDocNo'] = zotc_clean['AccDocNo'].apply(extract_scalar_acc_doc_no)

# # Drop rows where AccDocNo couldn't be resolved
# zotc_clean = zotc_clean.dropna(subset=['AccDocNo']).copy()

# # print("Unique types in AccDocNo:", zotc_clean['AccDocNo'].apply(type).unique())
# # print("Sample AccDocNo values:\n", zotc_clean['AccDocNo'].head(10))

Missing columns in ZOTC: ['OS Rep', 'Carrier Key Desc']
Unique types in AccDocNo: [<class 'str'>]
Sample AccDocNo values:
 0    9905683460.0
1    9905683460.0
2    9905683460.0
3    9905683480.0
4    9905683483.0
5    9905683485.0
6    9905683485.0
7    9905683487.0
8    9905683487.0
9    9905683526.0
Name: AccDocNo, dtype: object


In [43]:
zotc.columns

Index(['Sold-To Key', 'Sold-To Id', 'Document Currency', 'Net Product Price',
       'Net Sales', 'Sales Cost', 'ACOP Amount', 'Net Sales Cost',
       'Sales Margin', 'Sales Margin %', 'Quantity Sold', 'Invoice Number',
       'Invoice Line Item', 'Invoice Date', 'Order Number', 'Order Date',
       'Purchase Order Type Desc', 'Order Line Item', 'Billing Type',
       'Document Type Key', 'IS Rep', 'Ship-to Key', 'Ship-to',
       'Customer PO Number', 'Sales Office', 'Sales Group',
       'Terms Of Payment Desc', 'Material Key', 'Material Desc',
       'Manufacturer Part Number', 'Vendor Sub Range', 'Vendor Sub Range Desc',
       'Vendor Key', 'Vendor', 'Order Created By'],
      dtype='object')

In [54]:
import pandas as pd

def zotc_to_list_dict(zotc_df, invoice_col='Invoice Number', required_cols=None):
    """
    Manually aggregates ZOTC data: for each Invoice Number,
    returns a dict of {col: list of unique non-null values}.
    
    Safe even if invoice_col has messy types (lists, NaN, etc.).
    """
    if required_cols is None:
        required_cols = ['IS Rep', 'OS Rep', 'Carrier Key Desc', 'Customer PO Number', 'Sold-To Id']
    
    # Check columns exist
    missing = [col for col in required_cols if col not in zotc_df.columns]
    if missing:
        print(f"Missing in ZOTC: {missing}")
    for col in missing:
        required_cols.remove(col)
    
    result_list = []
    
    # Normalize invoice number extraction (safe for lists, NaN, etc.)
    def safe_scalar(x):
        if isinstance(x, list):
            for item in x:
                if pd.notna(item):
                    return str(item).strip()
            return None
        elif pd.isna(x):
            return None
        else:
            return str(x).strip()
    
    # Iterate over all rows and collect by AccDocNo
    accdoc_to_data = {}
    
    for _, row in zotc_df.iterrows():
        inv_num = row[invoice_col]
        key = safe_scalar(inv_num)
        if key is None:
            continue  # skip invalid keys
        
        # Initialize dict for this AccDocNo
        if key not in accdoc_to_data:
            accdoc_to_data[key] = {col: [] for col in required_cols}
        
        # Append non-null values from each required column
        for col in required_cols:
            val = row[col]
            if pd.notna(val):
                str_val = str(val).strip()
                if str_val not in accdoc_to_data[key][col]:  # avoid dups manually
                    accdoc_to_data[key][col].append(str_val)
    
    # Convert dict to list of dicts for DataFrame
    for accdocno, cols_dict in accdoc_to_data.items():
        record = {'AccDocNo': accdocno}
        for col, vals in cols_dict.items():
            record[col] = vals  # already a list
        result_list.append(record)
    
    return pd.DataFrame(result_list)

# ---------------------------------------------------
# 🚀 Use the function
# ---------------------------------------------------

# Clean GL table's AccDocNo similarly
def clean_acc_doc_no(x):
    if isinstance(x, list):
        for item in x:
            if pd.notna(item):
                return str(item).strip()
        return None
    elif pd.isna(x):
        return None
    else:
        return str(x).strip()

# Apply to GL
gl_clean = final_df.copy()
gl_clean['AccDocNo'] = final_df['AccDocNo'].apply(clean_acc_doc_no)
gl_clean = gl_clean.dropna(subset=['AccDocNo']).copy()
gl_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18992 entries, 0 to 18991
Data columns (total 64 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   FiscalYear                                  18992 non-null  int64         
 1   AccDocNo                                    18992 non-null  object        
 2   Dr_Cr                                       18992 non-null  object        
 3   DateOfAccDocEntry                           18992 non-null  object        
 4   DateOfAccDocEntry_and_TimeOfEntry (MYT)     18992 non-null  datetime64[ns]
 5   NoOfLineItemInAccDoc                        18992 non-null  object        
 6   RefKeyForLineItem                           18992 non-null  object        
 7   GL_DESCRIPTION                              18992 non-null  object        
 8   Amt_localCurrencyFormatted                  18992 non-null  object        
 9   Docume

In [55]:
# Apply manual aggregation to ZOTC
zotc_summarized = zotc_to_list_dict(
    zotc_df=zotc,
    invoice_col='Invoice Number',
    required_cols=['IS Rep', 'OS Rep', 'Carrier Key Desc', 'Customer PO Number', 'Sold-To Id']
)

zotc_summarized.info()

Missing in ZOTC: ['OS Rep', 'Carrier Key Desc']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1084687 entries, 0 to 1084686
Data columns (total 4 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   AccDocNo            1084687 non-null  object
 1   IS Rep              1084687 non-null  object
 2   Customer PO Number  1084687 non-null  object
 3   Sold-To Id          1084687 non-null  object
dtypes: object(4)
memory usage: 33.1+ MB


In [ ]:
for col in zotc_summarized.columns:
    agg_col(zotc_summarized, [col])

Aggregating AccDocNo
✅ Column 'AccDocNo' was successfully aggregated and flattened.

Aggregating IS Rep
❌ Column 'IS Rep' could not be fully aggregated. Keeping original.

Aggregating Customer PO Number
❌ Column 'Customer PO Number' could not be fully aggregated. Keeping original.

Aggregating Sold-To Id
✅ Column 'Sold-To Id' was successfully aggregated and flattened.



In [91]:
zotc_summarized.head(3)

,AccDocNo,IS Rep,Customer PO Number,Sold-To Id
0,9905683460.0,[Hui Mee Lau],[SW/CCJF/1116],ALPHA TELECOMMUNICATION SDN BHD
1,9905683480.0,[Pei Lian Kong],[P27066],EVOSS CONNECT SDN BHD
2,9905683483.0,[Wei Seng Teoh],[DATACASH/18072016],DATACASH ENTERPRISE SDN BHD


In [78]:
from pandas.api.types import infer_dtype

for col in zotc_summarized.select_dtypes(include='object').columns:
    print(f"{col}: {infer_dtype(zotc_summarized[col])}")

AccDocNo: string
IS Rep: mixed
Customer PO Number: mixed
Sold-To Id: mixed


In [47]:
# Now merge with GL
result = pd.merge(
    gl_clean,
    zotc_summarized,
    on='AccDocNo',
    how='left'
)

# Ensure all required columns are lists (in case of no match → fill with [])
# for col in zotc_summarized.columns:
    # if col != 'AccDocNo':
        # result[col] = result[col].apply(lambda x: x if isinstance(x, list) else [])

# ✅ Done!
# print(result[['AccDocNo', 'IS Rep', 'OS Rep']].head())
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18992 entries, 0 to 18991
Data columns (total 67 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   FiscalYear                                  18992 non-null  int64         
 1   AccDocNo                                    18992 non-null  object        
 2   Dr_Cr                                       18992 non-null  object        
 3   DateOfAccDocEntry                           18992 non-null  object        
 4   DateOfAccDocEntry_and_TimeOfEntry (MYT)     18992 non-null  datetime64[ns]
 5   NoOfLineItemInAccDoc                        18992 non-null  object        
 6   RefKeyForLineItem                           18992 non-null  object        
 7   GL_DESCRIPTION                              18992 non-null  object        
 8   Amt_localCurrencyFormatted                  18992 non-null  object        
 9   Docume

In [33]:
result.columns

Index(['FiscalYear', 'AccDocNo', 'Dr_Cr', 'DateOfAccDocEntry',
       'DateOfAccDocEntry_and_TimeOfEntry (MYT)', 'NoOfLineItemInAccDoc',
       'RefKeyForLineItem', 'GL_DESCRIPTION', 'Amt_localCurrencyFormatted',
       'Document Type Description', 'Quantity', 'Username', 'TransactionCode',
       'DocHeaderText', 'IsDocReversalOrReversedDoc', 'IdOfLineItem',
       'PostingKey', 'isLineItemAutomaticallyCreated', 'TermsOfPayment',
       'CashDiscountAmountInLocalCurrency', 'PaymentMethod', 'Plant',
       'hasSubsequentDrCrMemo', 'WBSElement', 'SpecialGLIndicator',
       'GL_AccType_AccountTypeDesc', 'Username Type',
       'Amt_DocCurrency_NoOfTrailingZeroes', 'CashDiscountPercentage',
       'IND_MonthEnd', 'IND_QuarterEnd', 'IND_YearEnd', 'IND_Weekend',
       'IND_PublicHol', 'IND_OutsideWorkHours', 'labelled_GL_AccType',
       'labelled_AccountTypeDesc', 'labelled_GL_AccType_AccountTypeDesc',
       'labelled_GL_DESCRIPTION', 'labelled_Document Type Description',
       'labell

In [37]:
result['IS Rep'].value_counts()

Series([], Name: count, dtype: int64)

In [38]:
if 'OS Rep' in result:
    # Suppose 'your_column' is the column of interest
    is_list_mask = result['OS Rep'].apply(lambda x: isinstance(x, list))

    # Now filter rows where the column is a list
    rows_with_lists = result[is_list_mask]
    # rows_with_lists

    # Extract the list values
    lists_only = result.loc[is_list_mask, 'OS Rep']

    # Get the lengths of those lists
    list_lengths = lists_only.apply(len)

In [ ]:
# result[result['AccDocNo']=='9910291310']  # 2018, more than 1 IS rep
# result[result['AccDocNo']=='9950469419']  # 2017, more than 1 IS rep

## SQL

In [ ]:
SELECT
    gl.*,
    STRING_AGG(zotc.[IS Rep], ',') WITHIN GROUP (ORDER BY zotc.[IS Rep]) AS [IS Rep List],
    STRING_AGG(zotc.[OS Rep], ',') WITHIN GROUP (ORDER BY zotc.[OS Rep]) AS [OS Rep List],
    STRING_AGG(zotc.[Carrier Key Desc], ',') WITHIN GROUP (ORDER BY zotc.[Carrier Key Desc]) AS [Carrier Key Desc List],
    STRING_AGG(zotc.[Customer PO Number], ',') WITHIN GROUP (ORDER BY zotc.[Customer PO Number]) AS [Customer PO Number List],
    STRING_AGG(zotc.[Sold-To Id], ',') WITHIN GROUP (ORDER BY zotc.[Sold-To Id]) AS [Sold-To Id List]
INTO
    data_ishi_GL_cleaned_with_ZOTC_{year}
FROM
    data_ishi_GL_cleaned_{year} gl
LEFT JOIN
    data_ishi_ZOTC_{year} zotc
    ON gl.AccDocNo = zotc.[Invoice Number]
GROUP BY
    gl.AccDocNo,
    -- Include all columns from GL table here:
    gl.[Col1], gl.[Col2], ... -- Replace with actual column names from GL_cleaned

In [ ]:
-- OLD SQL MERGING
SELECT 
    DISTINCT data_ishi_GL_cleaned_2019.*,
    data_ishi_ZOTC_2019.[Carrier Key Desc],
    data_ishi_ZOTC_2019.[Customer PO Number],
    data_ishi_ZOTC_2019.[IS Rep],
    data_ishi_ZOTC_2019.[Sold-To Id]
INTO 
    data_ishi_merged_2019
FROM 
    data_ishi_GL_cleaned_2019
LEFT JOIN 
    data_ishi_ZOTC_2019
    ON data_ishi_ZOTC_2019.[Invoice Number] = data_ishi_GL_cleaned_2019.AccDocNo;

In [ ]:
-- Alternative for SQL Server < 2017
SELECT
    gl.*,
    STUFF((
        SELECT ',' + z.[IS Rep]
        FROM data_ishi_ZOTC_{year} z
        WHERE z.[Invoice Number] = gl.AccDocNo
        GROUP BY z.[IS Rep]
        FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'), 1, 1, '') AS [IS Rep List],
    -- Repeat for other columns: OS Rep, Carrier Key Desc, etc.
INTO
    data_ishi_GL_cleaned_with_ZOTC_{year}
FROM
    data_ishi_GL_cleaned_{year} gl

# Save

In [ ]:
table_name="data_p_ishi_GL_JE_cleaned_SIEWPL"

try:
    sql_db = Conn_ODBC(database="JE_ML_2025")
    sql_db.fn_create_new_table_from_df(table_name=table_name, df=final_df, auto_data_type=True)
    res=sql_db.fn_append_df_to_table(table_name=table_name, df=final_df)
    print(res)
except Exception as e:
    print(e)
    # final_df.to_excel(rf"D:\victoriaquek\JE ML 2025\Results\{table_name}.xlsx")
    # final_df.to_csv(rf"D:\victoriaquek\JE ML 2025\Results\{table_name}.csv")
    final_df.to_csv(rf"D:\victoriaquek\JE ML 2025\Results\{table_name}")

# 20 mins

In [ ]:
result.info()

In [ ]:
table_name="data_p_ishi_GL_JE_cleaned_2019"

sql_db.fn_create_new_table_from_df(table_name=table_name, df=result, auto_data_type=True)
res=sql_db.fn_append_df_to_table(table_name=table_name, df=result)
print(res)

# 20 mins